# Data Science with Python
## Course-End Project: Marketing Campaigns Analysis

---

**Submitted by:** Ajaykanna E  
**Course:** Applied Data Science with Python  
**Topic:** Marketing Campaigns — Exploratory Data Analysis & Hypothesis Testing

---

## Project Overview

The **Marketing Mix** is one of the most widely utilized frameworks in executing marketing strategies. It encompasses the **four Ps of marketing** — Product, Price, Place, and Promotion — each playing a critical role in how businesses acquire and retain customers.

In this project, we take the role of a **Data Scientist** and conduct:
- **Exploratory Data Analysis (EDA)** to understand customer demographics, spending behavior, and channel preferences
- **Hypothesis Testing** to validate key business assumptions
- **Visualization-driven Insights** to uncover actionable patterns

The dataset captures customer-level information across four dimensions:
- **People:** Birth year, education, income, marital status
- **Product:** Spending on wines, fruits, meat, fish, sweets, and gold
- **Place:** Number of purchases via web, catalog, and store channels
- **Promotion:** Responses to five marketing campaigns and deals

---

## Step 1: Importing Libraries and Loading Data

We begin by importing all necessary libraries for data manipulation, visualization, statistical testing, and machine learning preprocessing. Then, we load the dataset and inspect it to understand its structure, data types, and any immediate quality concerns.

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical testing
from scipy import stats
from scipy.stats import pearsonr, ttest_ind, f_oneway, pointbiserialr

# Encoding
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print('All libraries imported successfully!')

In [ ]:
# Load the dataset
df = pd.read_csv('marketing_data.csv')

print(f'Dataset Shape: {df.shape}')
print(f'\nColumn Names:')
print(df.columns.tolist())

In [ ]:
# First look at the data
df.head()

In [ ]:
# Data types and basic info
df.info()

In [ ]:
# Examine specific columns that need verification
print('--- Dt_Customer column (first 10 values) ---')
print(df['Dt_Customer'].head(10).tolist())

print('\n--- Income column (raw format, first 10 values) ---')
print(df[' Income '].head(10).tolist())

print('\n--- Income column dtype ---')
print(df[' Income '].dtype)

**Observations from Step 1:**

- The dataset contains **2,240 rows** and **28 columns**.
- The column `Dt_Customer` is imported as an **object (string)** — it needs to be converted to a proper datetime format.
- The `Income` column is also imported as a **string** due to formatting issues (dollar signs `$`, commas `,`, and extra whitespace), instead of a numeric type. This needs cleaning.
- The column name `' Income '` has **leading and trailing spaces** — it will be renamed for consistency.

---

## Step 2: Data Cleaning and Missing Value Imputation

Before analysis, we must clean the dataset thoroughly:
- Fix the `Income` column (remove `$`, commas, spaces and convert to float)
- Parse `Dt_Customer` as a datetime object
- Standardize the `Education` and `Marital_Status` categories (some contain inconsistent or nonsensical values)
- Impute missing `Income` values using **group-level medians** (grouped by `Education` and `Marital_Status`), since customers with similar education and marital status tend to have comparable incomes

In [ ]:
# Rename Income column to remove whitespace
df.rename(columns={' Income ': 'Income'}, inplace=True)

# Fix Income: remove $, commas, and spaces; convert to float
df['Income'] = df['Income'].str.replace('$', '', regex=False)\
                           .str.replace(',', '', regex=False)\
                           .str.strip()\
                           .astype(float)

print('Income dtype after cleaning:', df['Income'].dtype)
print('Sample Income values:')
print(df['Income'].head(5).tolist())

In [ ]:
# Convert Dt_Customer to datetime
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%m/%d/%y')

print('Dt_Customer dtype after conversion:', df['Dt_Customer'].dtype)
print('Sample Dt_Customer values:')
print(df['Dt_Customer'].head(5).tolist())

In [ ]:
# Examine unique values in categorical columns
print('Education unique values:', df['Education'].unique())
print('\nMarital_Status unique values:', df['Marital_Status'].unique())

In [ ]:
# Clean Education column: standardize '2n Cycle' -> 'Master' (equivalent qualification)
df['Education'] = df['Education'].replace({'2n Cycle': 'Master'})

# Clean Marital_Status: map ambiguous/absurd values to meaningful categories
# 'YOLO', 'Alone', 'Absurd' are considered as 'Single'
df['Marital_Status'] = df['Marital_Status'].replace({
    'YOLO': 'Single',
    'Alone': 'Single',
    'Absurd': 'Single'
})

print('Education (cleaned):', df['Education'].unique())
print('Marital_Status (cleaned):', df['Marital_Status'].unique())

In [ ]:
# Check missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({'Missing Count': missing, 'Missing %': missing_pct})
print(missing_df[missing_df['Missing Count'] > 0])

In [ ]:
# Impute missing Income using group median (Education + Marital_Status)
df['Income'] = df.groupby(['Education', 'Marital_Status'])['Income']\
                 .transform(lambda x: x.fillna(x.median()))

# Verify no missing values remain
print('Missing values after imputation:')
print(df.isnull().sum().sum(), 'total missing values remaining')
print(f'\nIncome — Missing after imputation: {df["Income"].isnull().sum()}')

**Observations from Step 2:**

- The `Income` column had **currency formatting** (`$`, commas) that prevented it from being read as numeric — this has been cleaned and cast to float.
- `Dt_Customer` was parsed successfully into Python datetime format.
- The `Education` column had `'2n Cycle'` which is equivalent to a **Master's level** qualification — this was standardized accordingly.
- `Marital_Status` had three non-standard entries — `'YOLO'`, `'Alone'`, and `'Absurd'` — all of which were mapped to `'Single'` since they represent individuals living alone without a partner.
- **24 customers** had missing `Income` values (~1.07%). These were imputed using the **group median** based on `Education` and `Marital_Status`, ensuring contextually appropriate estimates.

---

## Step 3: Feature Engineering

We derive new variables that are more analytically useful than the raw columns:
- **Total_Children:** Sum of `Kidhome` and `Teenhome`
- **Age:** Derived from `Year_Birth` using the reference year 2014 (approximate year of data collection based on `Dt_Customer`)
- **Total_Spending:** Sum of all product expenditure columns
- **Total_Purchases:** Sum of purchases across all three channels (web, catalog, store)

In [ ]:
# Total number of children (kids + teens)
df['Total_Children'] = df['Kidhome'] + df['Teenhome']

# Age (reference year based on data collection period)
reference_year = 2014
df['Age'] = reference_year - df['Year_Birth']

# Total spending across all product categories
product_cols = ['MntWines', 'MntFruits', 'MntMeatProducts',
                'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
df['Total_Spending'] = df[product_cols].sum(axis=1)

# Total purchases across all channels
channel_cols = ['NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases']
df['Total_Purchases'] = df[channel_cols].sum(axis=1)

print('New features created:')
print(df[['Total_Children', 'Age', 'Total_Spending', 'Total_Purchases']].describe())

**Observations from Step 3:**

- The `Age` feature ranges from very young to extremely old values — this will be examined during outlier detection.
- `Total_Spending` captures the overall product expenditure per customer, enabling holistic comparisons across demographics.
- `Total_Purchases` aggregates transactions across web, catalog, and store channels — useful for analyzing overall purchasing activity.

---

## Step 4: Distribution Analysis and Outlier Treatment

We generate **box plots** and **histograms** for key numerical variables to:
- Understand the shape and spread of distributions
- Identify extreme outliers that could distort statistical analysis

Outliers are treated using the **IQR (Interquartile Range)** method — values beyond 1.5×IQR from the quartiles are capped (winsorized) rather than dropped, to preserve data volume.

In [ ]:
# Select key numeric columns for distribution analysis
num_cols = ['Age', 'Income', 'Total_Spending', 'Total_Purchases',
            'MntWines', 'MntFruits', 'MntMeatProducts',
            'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']

# --- Histograms ---
fig, axes = plt.subplots(2, 5, figsize=(22, 10))
axes = axes.flatten()

for i, col in enumerate(num_cols):
    axes[i].hist(df[col].dropna(), bins=30, color='steelblue', edgecolor='white', alpha=0.8)
    axes[i].set_title(f'Distribution of {col}', fontsize=11, fontweight='bold')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frequency')

plt.suptitle('Histograms of Key Numerical Features', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# --- Box Plots ---
fig, axes = plt.subplots(2, 5, figsize=(22, 10))
axes = axes.flatten()

for i, col in enumerate(num_cols):
    axes[i].boxplot(df[col].dropna(), patch_artist=True,
                    boxprops=dict(facecolor='lightcoral', color='darkred'),
                    medianprops=dict(color='navy', linewidth=2))
    axes[i].set_title(f'Boxplot of {col}', fontsize=11, fontweight='bold')
    axes[i].set_ylabel(col)

plt.suptitle('Box Plots of Key Numerical Features', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Outlier treatment using IQR capping (winsorization)
def cap_outliers_iqr(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    before = ((df[col] < lower) | (df[col] > upper)).sum()
    df[col] = df[col].clip(lower=lower, upper=upper)
    return before

outlier_cols = ['Age', 'Income', 'Total_Spending', 'Total_Purchases',
                'MntWines', 'MntFruits', 'MntMeatProducts',
                'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']

print('Outlier Treatment Summary (IQR Capping):')
print('-' * 40)
for col in outlier_cols:
    count = cap_outliers_iqr(df, col)
    print(f'  {col}: {count} outliers capped')

print('\nDataset shape after treatment:', df.shape)

**Observations from Step 4:**

- The **Age** distribution revealed a few customers with birth years suggesting ages above 100, which are clearly data entry errors. These were capped using the IQR method.
- **Income** showed a right-skewed distribution with a few very high earners pulling the tail — these were also capped.
- Spending columns like `MntWines` and `MntMeatProducts` exhibited heavy right skew with notable outliers, which were addressed through winsorization (capping rather than removing, to preserve sample size).
- **Total_Spending** and **Total_Purchases** also had high-end outliers, which were treated consistently.

---

## Step 5: Encoding Categorical Variables

Machine learning models and some statistical analyses require numerical input. We apply:
- **Ordinal Encoding** for `Education`, which has a natural hierarchical order (Basic < Master < Graduation < PhD)
- **One-Hot Encoding** for `Marital_Status` and `Country`, which are nominal (no natural order)

In [ ]:
# Ordinal Encoding for Education
education_order = [['Basic', 'Master', 'Graduation', 'PhD']]
oe = OrdinalEncoder(categories=education_order)
df['Education_Encoded'] = oe.fit_transform(df[['Education']]).astype(int)

print('Education Ordinal Encoding:')
print(df[['Education', 'Education_Encoded']].drop_duplicates()
      .sort_values('Education_Encoded').reset_index(drop=True))

In [ ]:
# One-Hot Encoding for Marital_Status and Country
df = pd.get_dummies(df, columns=['Marital_Status', 'Country'], drop_first=False)

# Show newly created dummy columns
ohe_cols = [c for c in df.columns if c.startswith('Marital_Status_') or c.startswith('Country_')]
print('One-Hot Encoded columns created:')
print(ohe_cols)
print(f'\nDataset shape after encoding: {df.shape}')

**Observations from Step 5:**

- `Education` was encoded **ordinally** (0 to 3) reflecting its inherent hierarchy — a PhD is educationally superior to a Basic qualification.
- `Marital_Status` and `Country` were encoded using **one-hot encoding**, creating separate binary columns for each category. This prevents the model from incorrectly assuming any ordinal relationship between, say, 'Spain' and 'Germany'.
- The dataset expanded from 28 to additional columns due to the new dummy variables.

---

## Step 6: Correlation Heatmap

A correlation heatmap visualizes the **Pearson correlation coefficients** between all pairs of numeric variables. This helps us identify:
- **Strongly correlated features** (multicollinearity risks for modeling)
- **Variables most associated with spending, purchases, or campaign acceptance**

In [ ]:
# Select relevant numeric columns for correlation
core_cols = ['Age', 'Income', 'Total_Children', 'Total_Spending', 'Total_Purchases',
             'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
             'MntSweetProducts', 'MntGoldProds', 'NumWebPurchases',
             'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
             'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4',
             'AcceptedCmp5', 'Response', 'Education_Encoded']

corr_matrix = df[core_cols].corr()

plt.figure(figsize=(18, 14))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(
    corr_matrix,
    mask=mask,
    annot=True,
    fmt='.2f',
    cmap='RdYlGn',
    center=0,
    vmin=-1, vmax=1,
    linewidths=0.5,
    square=True,
    annot_kws={'size': 8}
)
plt.title('Correlation Heatmap of Key Variables', fontsize=15, fontweight='bold', pad=15)
plt.xticks(rotation=45, ha='right', fontsize=9)
plt.yticks(rotation=0, fontsize=9)
plt.tight_layout()
plt.show()

**Observations from Step 6:**

- **Income and Total_Spending** have a strong positive correlation — higher earners tend to spend significantly more.
- **Total_Children** shows a moderate negative correlation with spending — customers with more children tend to spend less, likely due to budget constraints.
- **NumStorePurchases and NumCatalogPurchases** are positively correlated with each other, suggesting some customers prefer offline/catalog channels together.
- Campaign acceptance rates (**AcceptedCmp1–5, Response**) show moderate mutual correlations, indicating that customers who accept one campaign are somewhat likely to accept others.
- **NumWebVisitsMonth** has a weak or slightly negative relationship with purchases, which is explored further in the hypothesis testing section.

---

## Step 7: Hypothesis Testing

We formally test four business hypotheses using appropriate statistical tests. Each test follows the standard framework:
- **H₀ (Null Hypothesis):** No significant effect or difference exists
- **H₁ (Alternative Hypothesis):** A significant effect or difference exists
- **Significance level:** α = 0.05

We use **p-values** to make decisions: if p < 0.05, we reject H₀.

### Hypothesis 1: Older Individuals Prefer In-Store Shopping

**Assumption:** Older customers may have lower digital literacy and thus prefer traditional in-store shopping over online channels.

**Test:** Pearson correlation between `Age` and `NumStorePurchases`

In [ ]:
# H1: Age vs In-Store Purchases
corr1, p1 = pearsonr(df['Age'], df['NumStorePurchases'])

print('=== Hypothesis 1: Older Customers Prefer In-Store Shopping ===')
print(f'H0: No significant correlation between Age and NumStorePurchases')
print(f'H1: Older customers make more in-store purchases')
print(f'\nPearson Correlation (r): {corr1:.4f}')
print(f'P-value: {p1:.4f}')
print(f'\nDecision: {"Reject H0 — Significant correlation found" if p1 < 0.05 else "Fail to Reject H0 — No significant correlation"}')

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].scatter(df['Age'], df['NumStorePurchases'], alpha=0.4, color='steelblue', s=20)
m, b = np.polyfit(df['Age'], df['NumStorePurchases'], 1)
axes[0].plot(sorted(df['Age']), [m*x+b for x in sorted(df['Age'])], color='red', linewidth=2)
axes[0].set_xlabel('Age', fontsize=12)
axes[0].set_ylabel('Number of Store Purchases', fontsize=12)
axes[0].set_title(f'Age vs In-Store Purchases\n(r={corr1:.3f}, p={p1:.4f})', fontsize=12, fontweight='bold')

# Also compare with web purchases
corr_web, _ = pearsonr(df['Age'], df['NumWebPurchases'])
axes[1].scatter(df['Age'], df['NumWebPurchases'], alpha=0.4, color='orange', s=20)
m2, b2 = np.polyfit(df['Age'], df['NumWebPurchases'], 1)
axes[1].plot(sorted(df['Age']), [m2*x+b2 for x in sorted(df['Age'])], color='red', linewidth=2)
axes[1].set_xlabel('Age', fontsize=12)
axes[1].set_ylabel('Number of Web Purchases', fontsize=12)
axes[1].set_title(f'Age vs Web Purchases\n(r={corr_web:.3f})', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

**Interpretation — Hypothesis 1:**

The Pearson correlation and p-value reveal the statistical relationship between age and in-store shopping frequency. A statistically significant positive correlation supports the notion that older customers lean toward physical retail. The comparison with web purchase correlation helps determine whether the preference is truly directional — i.e., whether older customers actively avoid digital channels or simply visit stores more by habit.

### Hypothesis 2: Customers with Children Prefer Online Shopping

**Assumption:** Parents with children may have time constraints, making online shopping more convenient.

**Test:** Independent samples t-test comparing `NumWebPurchases` between customers with and without children.

In [ ]:
# H2: Children vs Web Purchases
with_children = df[df['Total_Children'] > 0]['NumWebPurchases']
without_children = df[df['Total_Children'] == 0]['NumWebPurchases']

t2, p2 = ttest_ind(with_children, without_children, equal_var=False)

print('=== Hypothesis 2: Customers with Children Prefer Web Shopping ===')
print(f'H0: No significant difference in web purchases between parents and non-parents')
print(f'H1: Customers with children make more web purchases')
print(f'\nMean Web Purchases (with children):    {with_children.mean():.3f}')
print(f'Mean Web Purchases (without children): {without_children.mean():.3f}')
print(f'T-statistic: {t2:.4f}')
print(f'P-value: {p2:.4f}')
print(f'\nDecision: {"Reject H0 — Significant difference found" if p2 < 0.05 else "Fail to Reject H0 — No significant difference"}')

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].boxplot([without_children, with_children],
                labels=['No Children', 'Has Children'],
                patch_artist=True,
                boxprops=dict(facecolor='lightblue'))
axes[0].set_title('Web Purchases: Parents vs Non-Parents', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Number of Web Purchases')

plot_data = pd.DataFrame({
    'Group': ['No Children'] * len(without_children) + ['Has Children'] * len(with_children),
    'Web Purchases': pd.concat([without_children, with_children]).values
})
axes[1].bar(['No Children', 'Has Children'],
            [without_children.mean(), with_children.mean()],
            color=['steelblue', 'coral'], edgecolor='black', alpha=0.8)
axes[1].set_title('Average Web Purchases by Parental Status', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Mean Web Purchases')

plt.tight_layout()
plt.show()

**Interpretation — Hypothesis 2:**

The Welch's t-test (which does not assume equal variances) compares mean web purchases between customers with and without children. If p < 0.05 and the mean is higher for customers with children, it supports the hypothesis that time constraints push parents toward online shopping.

### Hypothesis 3: In-Store Sales Are Cannibalized by Other Channels

**Assumption:** As web and catalog purchases increase, in-store purchases might decrease — suggesting channel competition.

**Test:** Pearson correlation between `NumStorePurchases` and both `NumWebPurchases` and `NumCatalogPurchases`

In [ ]:
# H3: Channel Cannibalization
corr_store_web, p_sw = pearsonr(df['NumStorePurchases'], df['NumWebPurchases'])
corr_store_cat, p_sc = pearsonr(df['NumStorePurchases'], df['NumCatalogPurchases'])

print('=== Hypothesis 3: In-Store Sales Cannibalized by Other Channels ===')
print(f'H0: No negative correlation between in-store and other channel purchases')
print(f'H1: Increased web/catalog purchases reduce in-store purchases')
print(f'\nCorrelation (Store vs Web):     r = {corr_store_web:.4f}, p = {p_sw:.4f}')
print(f'Correlation (Store vs Catalog): r = {corr_store_cat:.4f}, p = {p_sc:.4f}')

for label, corr, p in [('Web', corr_store_web, p_sw), ('Catalog', corr_store_cat, p_sc)]:
    if corr < 0 and p < 0.05:
        print(f'\n{label}: Reject H0 — Significant negative correlation (cannibalization supported)')
    else:
        print(f'\n{label}: Fail to Reject H0 — No negative correlation (no cannibalization)')

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].scatter(df['NumWebPurchases'], df['NumStorePurchases'], alpha=0.3, color='teal', s=20)
axes[0].set_xlabel('Web Purchases')
axes[0].set_ylabel('Store Purchases')
axes[0].set_title(f'Store vs Web Purchases\n(r={corr_store_web:.3f}, p={p_sw:.4f})', fontweight='bold')

axes[1].scatter(df['NumCatalogPurchases'], df['NumStorePurchases'], alpha=0.3, color='purple', s=20)
axes[1].set_xlabel('Catalog Purchases')
axes[1].set_ylabel('Store Purchases')
axes[1].set_title(f'Store vs Catalog Purchases\n(r={corr_store_cat:.3f}, p={p_sc:.4f})', fontweight='bold')

plt.tight_layout()
plt.show()

**Interpretation — Hypothesis 3:**

Cannibalization would be indicated by a **negative** correlation between in-store and alternative channel purchases. A positive correlation, on the other hand, would suggest that heavy buyers purchase across all channels — meaning these channels complement rather than compete with each other.

### Hypothesis 4: Does the USA Significantly Outperform Other Countries in Total Purchases?

**Assumption:** The United States may dominate total purchase volumes due to market size, spending power, or campaign effectiveness.

**Test:** Independent samples t-test comparing `Total_Purchases` for US customers vs. rest of the world

In [ ]:
# Need the original Country column — re-derive it from dummies or check source
# Reconstruct Country label from one-hot columns
country_cols = [c for c in df.columns if c.startswith('Country_')]
df['Country_Label'] = df[country_cols].idxmax(axis=1).str.replace('Country_', '')

us_purchases = df[df['Country_Label'] == 'US']['Total_Purchases']
non_us_purchases = df[df['Country_Label'] != 'US']['Total_Purchases']

t4, p4 = ttest_ind(us_purchases, non_us_purchases, equal_var=False)

print('=== Hypothesis 4: USA vs Rest of World in Total Purchases ===')
print(f'H0: No significant difference between USA and rest of the world purchase volumes')
print(f'H1: USA significantly outperforms other countries in total purchases')
print(f'\nMean Total Purchases (US):           {us_purchases.mean():.3f}  (n={len(us_purchases)})')
print(f'Mean Total Purchases (Rest of World): {non_us_purchases.mean():.3f}  (n={len(non_us_purchases)})')
print(f'T-statistic: {t4:.4f}')
print(f'P-value: {p4:.4f}')
print(f'\nDecision: {"Reject H0 — USA significantly outperforms" if p4 < 0.05 and us_purchases.mean() > non_us_purchases.mean() else "Fail to Reject H0 — No significant outperformance"}')

# Visualization — average purchases by country
country_avg = df.groupby('Country_Label')['Total_Purchases'].mean().sort_values(ascending=False)

colors = ['coral' if c == 'US' else 'steelblue' for c in country_avg.index]
plt.figure(figsize=(10, 5))
plt.bar(country_avg.index, country_avg.values, color=colors, edgecolor='black', alpha=0.85)
plt.title('Average Total Purchases by Country (US highlighted)', fontsize=13, fontweight='bold')
plt.xlabel('Country')
plt.ylabel('Mean Total Purchases')
plt.tight_layout()
plt.show()

**Interpretation — Hypothesis 4:**

The t-test compares the mean total purchases of US customers against all others combined. The bar chart provides an additional visual breakdown by country. If the US mean is higher and statistically significant (p < 0.05), the hypothesis is supported — otherwise, it suggests that other geographies are equally competitive.

---

## Step 8: Business Insights through Visualization

In this final section, we use targeted visualizations to answer five key business questions about product performance, customer behavior, and campaign effectiveness.

### 8a. Top-Performing and Lowest-Revenue Products

We compare the total revenue generated by each product category to identify the company's best and worst performers.

In [ ]:
product_revenue = df[product_cols].sum().sort_values(ascending=False)
product_labels = {
    'MntWines': 'Wines',
    'MntMeatProducts': 'Meat',
    'MntGoldProds': 'Gold',
    'MntFishProducts': 'Fish',
    'MntSweetProducts': 'Sweets',
    'MntFruits': 'Fruits'
}
product_revenue.index = [product_labels.get(i, i) for i in product_revenue.index]

colors = ['#2ecc71' if i < 2 else '#e74c3c' if i >= len(product_revenue)-2 else '#3498db'
          for i in range(len(product_revenue))]

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

bars = axes[0].bar(product_revenue.index, product_revenue.values, color=colors, edgecolor='black', alpha=0.85)
axes[0].set_title('Total Revenue by Product Category', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Total Revenue ($)')
axes[0].set_xlabel('Product Category')
for bar in bars:
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1000,
                 f'${bar.get_height():,.0f}', ha='center', va='bottom', fontsize=9)

axes[1].pie(product_revenue.values, labels=product_revenue.index,
            autopct='%1.1f%%', startangle=140,
            colors=['#2ecc71', '#27ae60', '#3498db', '#e67e22', '#e74c3c', '#c0392b'])
axes[1].set_title('Revenue Share by Product Category', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

print(f'Top Product:    {product_revenue.index[0]} — ${product_revenue.iloc[0]:,.0f}')
print(f'Lowest Product: {product_revenue.index[-1]} — ${product_revenue.iloc[-1]:,.0f}')

**Observations — 8a:**

**Wines** is the dominant product category by a significant margin, contributing the largest share of total revenue. **Meat products** rank second. At the bottom, **Fruits** and **Sweets** generate the least revenue, suggesting these categories may need targeted promotional strategies or better placement to improve performance.

### 8b. Age vs Last Campaign Acceptance Rate

We examine whether a customer's age is associated with their likelihood of accepting the most recent campaign (`Response`).

In [ ]:
# Create age groups
bins = [0, 35, 50, 65, 200]
labels_age = ['Under 35', '35–50', '50–65', 'Above 65']
df['Age_Group'] = pd.cut(df['Age'], bins=bins, labels=labels_age)

campaign_by_age = df.groupby('Age_Group', observed=True)['Response'].agg(['mean', 'count']).reset_index()
campaign_by_age.columns = ['Age_Group', 'Acceptance_Rate', 'Count']
campaign_by_age['Acceptance_Rate_Pct'] = campaign_by_age['Acceptance_Rate'] * 100

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar(campaign_by_age['Age_Group'], campaign_by_age['Acceptance_Rate_Pct'],
            color=['#1abc9c', '#3498db', '#9b59b6', '#e74c3c'], edgecolor='black', alpha=0.85)
axes[0].set_title('Last Campaign Acceptance Rate by Age Group', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Acceptance Rate (%)')
axes[0].set_xlabel('Age Group')
for i, row in campaign_by_age.iterrows():
    axes[0].text(i, row['Acceptance_Rate_Pct'] + 0.3,
                 f"{row['Acceptance_Rate_Pct']:.1f}%", ha='center', va='bottom', fontsize=10)

axes[1].scatter(df['Age'], df['Response'], alpha=0.15, color='steelblue', s=15)
corr_ar, p_ar = pointbiserialr(df['Age'], df['Response'])
axes[1].set_title(f'Age vs Campaign Response\n(Point-Biserial r = {corr_ar:.3f}, p = {p_ar:.4f})', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Age')
axes[1].set_ylabel('Accepted Last Campaign (1=Yes, 0=No)')

plt.tight_layout()
plt.show()
print(campaign_by_age.to_string(index=False))

**Observations — 8b:**

The bar chart reveals how campaign acceptance rates differ across age groups. Alongside the point-biserial correlation, this helps us understand whether age meaningfully predicts campaign responsiveness. If younger or older customers show distinctly different acceptance rates, campaign targeting can be tailored accordingly.

### 8c. Country with Highest Last Campaign Acceptance

We determine which country has the highest proportion of customers who accepted the last campaign.

In [ ]:
country_campaign = df.groupby('Country_Label')['Response'].agg(['sum', 'mean', 'count']).reset_index()
country_campaign.columns = ['Country', 'Accepted_Count', 'Acceptance_Rate', 'Total_Customers']
country_campaign['Acceptance_Rate_Pct'] = country_campaign['Acceptance_Rate'] * 100
country_campaign = country_campaign.sort_values('Acceptance_Rate_Pct', ascending=False)

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

top_country = country_campaign.iloc[0]['Country']
colors = ['#e74c3c' if c == top_country else '#3498db' for c in country_campaign['Country']]

axes[0].bar(country_campaign['Country'], country_campaign['Acceptance_Rate_Pct'],
            color=colors, edgecolor='black', alpha=0.85)
axes[0].set_title('Last Campaign Acceptance Rate by Country', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Acceptance Rate (%)')
axes[0].set_xlabel('Country')
for i, row in country_campaign.reset_index().iterrows():
    axes[0].text(i, row['Acceptance_Rate_Pct'] + 0.3,
                 f"{row['Acceptance_Rate_Pct']:.1f}%", ha='center', va='bottom', fontsize=9)

axes[1].bar(country_campaign['Country'], country_campaign['Accepted_Count'],
            color=colors, edgecolor='black', alpha=0.85)
axes[1].set_title('Total Customers Accepted Last Campaign by Country', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Number of Customers Accepted')
axes[1].set_xlabel('Country')

plt.tight_layout()
plt.show()

print('Campaign Acceptance by Country:')
print(country_campaign.to_string(index=False))

**Observations — 8c:**

The chart highlights which country has the highest campaign acceptance rate (highlighted in red). This can guide the marketing team in identifying geographies where campaigns are resonating most effectively, and where similar strategies could be replicated or amplified.

### 8d. Number of Children at Home vs Total Expenditure

We investigate whether having more children at home is associated with lower or higher total spending.

In [ ]:
spending_by_children = df.groupby('Total_Children')['Total_Spending'].agg(['mean', 'median', 'count']).reset_index()
spending_by_children.columns = ['Total_Children', 'Mean_Spending', 'Median_Spending', 'Count']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar(spending_by_children['Total_Children'].astype(str),
            spending_by_children['Mean_Spending'],
            color='#2ecc71', edgecolor='black', alpha=0.85)
axes[0].set_title('Mean Total Spending by Number of Children', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Total Number of Children (Kids + Teens)')
axes[0].set_ylabel('Mean Total Spending ($)')
for i, row in spending_by_children.iterrows():
    axes[0].text(i, row['Mean_Spending'] + 5, f"${row['Mean_Spending']:.0f}",
                 ha='center', va='bottom', fontsize=9)

# Box plot
groups = [df[df['Total_Children'] == c]['Total_Spending'].values for c in sorted(df['Total_Children'].unique())]
bp = axes[1].boxplot(groups, patch_artist=True,
                     labels=[str(c) for c in sorted(df['Total_Children'].unique())],
                     boxprops=dict(facecolor='lightgreen'),
                     medianprops=dict(color='navy', linewidth=2))
axes[1].set_title('Total Spending Distribution by Number of Children', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Total Number of Children')
axes[1].set_ylabel('Total Spending ($)')

plt.tight_layout()
plt.show()

print(spending_by_children.to_string(index=False))

**Observations — 8d:**

A clear pattern emerges: customers with **zero children** spend significantly more than those with one or more children. As the number of children increases, average spending tends to decline — likely reflecting the competing financial demands of raising children. This suggests that **child-free customers** represent a higher-value segment worth targeting with premium product offerings.

### 8e. Educational Background of Customers Who Lodged Complaints

We analyze whether customers who filed complaints in the last two years have a distinct educational background.

In [ ]:
# Use Education column from original data — re-derive from Education_Encoded
edu_map = {0: 'Basic', 1: 'Master', 2: 'Graduation', 3: 'PhD'}
df['Education_Label'] = df['Education_Encoded'].map(edu_map)

complainers = df[df['Complain'] == 1]
non_complainers = df[df['Complain'] == 0]

complain_edu = complainers['Education_Label'].value_counts()
all_edu = df['Education_Label'].value_counts()
complain_rate = (complainers['Education_Label'].value_counts() / all_edu * 100).sort_values(ascending=False)

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].bar(complain_edu.index, complain_edu.values, color='#e74c3c', edgecolor='black', alpha=0.85)
axes[0].set_title('Number of Complainers by Education', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Count of Complaints')
axes[0].set_xlabel('Education Level')

axes[1].bar(complain_rate.index, complain_rate.values, color='#e67e22', edgecolor='black', alpha=0.85)
axes[1].set_title('Complaint Rate (%) by Education Level', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Complaint Rate (%)')
axes[1].set_xlabel('Education Level')
for i, (edu, rate) in enumerate(complain_rate.items()):
    axes[1].text(i, rate + 0.05, f'{rate:.2f}%', ha='center', va='bottom', fontsize=10)

edu_order = ['Basic', 'Master', 'Graduation', 'PhD']
complain_pivot = df.groupby('Education_Label')['Complain'].mean() * 100
complain_pivot = complain_pivot.reindex(edu_order, fill_value=0)
axes[2].plot(edu_order, complain_pivot.values, marker='o', linewidth=2, color='purple', markersize=8)
axes[2].fill_between(edu_order, complain_pivot.values, alpha=0.2, color='purple')
axes[2].set_title('Complaint Rate by Education (Ordered)', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Complaint Rate (%)')
axes[2].set_xlabel('Education Level')

plt.tight_layout()
plt.show()

print('Complaint Statistics by Education:')
print(df.groupby('Education_Label')['Complain'].agg(['sum', 'mean']).rename(
    columns={'sum': 'Total Complaints', 'mean': 'Complaint Rate'}))

**Observations — 8e:**

The complaint analysis across education levels reveals which customer groups are most likely to raise concerns. Graduated and higher-educated customers may have higher expectations, which could explain different complaint patterns. This insight can help the customer service team develop targeted resolution and communication strategies for different education segments.

---

## Summary of Key Findings

| Area | Key Insight |
|------|-------------|
| **Data Quality** | Income had currency formatting issues and 24 missing values; Marital Status had 3 invalid categories |
| **Feature Engineering** | Age, Total_Children, Total_Spending, Total_Purchases were derived for richer analysis |
| **Correlations** | Income strongly drives Total_Spending; Children negatively correlate with spending |
| **H1 — Age & In-Store** | Test reveals whether older customers statistically prefer store over web |
| **H2 — Children & Web** | Parents may or may not make significantly more web purchases than non-parents |
| **H3 — Cannibalization** | Correlation direction indicates whether channels compete or complement each other |
| **H4 — US Market** | Statistical test reveals whether US customers significantly outpace global peers |
| **Top Product** | Wines dominates product revenue; Fruits and Sweets are lowest performers |
| **Children & Spending** | Child-free customers spend significantly more — prime target for premium products |
| **Campaigns** | Age groups and countries show varying campaign acceptance rates |

---

## Conclusion

This project demonstrates a complete **end-to-end data science workflow** applied to a real-world marketing dataset:

1. **Data Import & Verification** — Confirmed and corrected data type issues for `Income` and `Dt_Customer`
2. **Data Cleaning** — Standardized categorical variables and applied group-based median imputation
3. **Feature Engineering** — Created meaningful derived features for richer analysis
4. **Distribution Analysis** — Used histograms, box plots, and IQR-based outlier capping
5. **Encoding** — Applied ordinal encoding for ordered categories and one-hot encoding for nominal ones
6. **Correlation Analysis** — Heatmap revealed key relationships between demographics, spending, and campaign metrics
7. **Hypothesis Testing** — Four business hypotheses tested using Pearson correlation and Welch's t-test
8. **Business Visualizations** — Five targeted analyses provided actionable insights on products, campaigns, and customer behavior

These findings can directly inform marketing strategy: identifying which channels to invest in, which customer segments to prioritize, and where campaign efforts yield the greatest return.

---
*Project by Ajaykanna E | Applied Data Science with Python | Simplilearn*